In [6]:
# torch 는 딥러닝을 구성하기 편하도록 페이스북에서 만든 딥러닝 라이브러리. 텐서플로우랑 똑같은 딥러닝 라이브러리.
import torch

# 신경망 구축을 쉽게 해주는 함수.
import torch.nn as nn

In [7]:
# GPU 확인  GPU가 없으면 cpu로 연산 , GPU가 있으면 GPU로 연산하겠다는 준비  따라서 device 는 cpu나 gpu 문자열로 바뀐다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [8]:
# 4명의 데이터가 있는데, 1명당 2개의 컬럼을 가지고 있다. 그리고 그걸 2차원 리스트로 4명의 데이터를 선언 후 
#토치가 알아 먹을 수 있는 텐서타입으로 바꾼다. 이때 텐서타입은 float 
# to(device)는 모델이 gpu에서 연산될 것이기 때문에 to(cuda)를 함으로써 데이터를 gpu에 얹혀 놓는다.
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [9]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
           #batch normalization
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)

In [10]:
criterion = torch.nn.MSELoss().to(device) #Mean Square Error
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # 옵티마이저인 Stochestic Gradient Descent 를 쓸거다. 업데이트 대상은 model 객체가 가지고 있는
                                                          # W 들을 대상으로 업데이트를 할 것이고, learning rate 는 0.001로 설정할 것이다.

In [11]:
for epoch in range(50): # 50번 학습할거다.
  optimizer.zero_grad() # 옵티마이저의 기울기를 매번 초기화 해서 계산해야 하기 때문에 초기화 하는 함수를 선언
  # forward 연산
  hypothesis = model(X) # 모델에 데이터를 넣고, propagation을 진행한 후 , 최종 아웃풋을 hypothesis 에 담는다.

  #비율 함수
  cost = criterion(hypothesis, Y) # 예측값과 정답값 사이의 Mean Square Loss 를 구함
  cost.backward() # backpropagation이 아니다. 이름에 낚이지 말자. 이 부분은 기울기를 구하는 함수다
  optimizer.step() # 여기서 비로소 backpropagation을 실행하여 모델의 미ㅗ든 w들을 업데이트 한다

  # 5의 배수에 해당되는 에포크마다 비용을 출력력
  if epoch % 5 == 0:
        print("에폭:", epoch, cost.item())


에폭: 0 0.2604835629463196
에폭: 5 0.26044291257858276
에폭: 10 0.2604024410247803
에폭: 15 0.26036202907562256
에폭: 20 0.26032179594039917
에폭: 25 0.2602817416191101
에폭: 30 0.2602418065071106
에폭: 35 0.260202020406723
에폭: 40 0.2601623833179474
에폭: 45 0.2601228952407837


In [12]:
X_2 = torch.FloatTensor([[0, 0], [0, 1]]).to(device)
hypothesis = model(X_2)   # fitting 된 모델에 데이터를 넣어서 inference 하는 부분
print(hypothesis)

tensor([[0.3997],
        [0.4000]], device='cuda:0', grad_fn=<SigmoidBackward0>)


# **minibatch DNN**

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [38]:
# GPU 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [37]:
# 5명의 데이터, 각 데이터는 3개의 칼럼(피처)를 갖는다.
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
# 정답값은 1사람당 1개
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [36]:
# X 데이터셋과 Y 데이터 셋을 포장함함
dataset = TensorDataset(x_train, y_train)
print(list(dataset))

[(tensor([93., 88., 93.], device='cuda:0'), tensor([185.])), (tensor([89., 91., 90.], device='cuda:0'), tensor([180.]))]


In [35]:
# 2 개의 데이터를 1 세트로 만들어서 전체데이터셋을 쪼갠다. 이때 만든 n 개의 세트들의 순서를 shuffle 한다.
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [34]:
model = nn.Sequential(
          nn.Linear(3, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.ReLU(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.ReLU()
          ).to(device)

In [33]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 0.00001

In [39]:
nb_epochs = 5
for epoch in range(nb_epochs + 1): #전체 데이터 셋으로 5번 학습할 것이다.
  for batch_idx, samples in enumerate(dataloader): #minibatch 데이터를 반환하면서 index도 반환한다. GPU 터질까봐 mini batch 한다.

    x_train, y_train = samples
    x_train= x_train.to(device)
    y_train= y_train.to(y_train)

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = criterion(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

RuntimeError: ignored

In [40]:
X_2 = torch.FloatTensor([[100, 70, 60], [150, 30,75]]).to(device)
hypothesis = model(X_2)
print(hypothesis)

tensor([[0.9190],
        [2.4369]], device='cuda:0', grad_fn=<ReluBackward0>)


# **CNN**

In [41]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

In [43]:
learning_rate = 0.001 # 옵티마이저의 learning rate
training_epochs = 15 # 학습횟수
batch_size = 100 # minibatch 사이즈, 전체 데이터셋에서 서브 셋을 몇개로 묶을거냐

In [44]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환, 토치에서 알아듣는 타입으로 지정
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환, 토치에서 알아듣는 타입으로 지정
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 106943590.94it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 96522465.20it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26681885.65it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3149368.29it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [45]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True) #drop_last 마지막 남은 데이터를 버릴거냐 안버릴거냐

In [46]:
# torch.nn.Module:  PyTorch의 모든 Neural Network의 Base Class
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64) 
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [47]:
# CNN 모델 정의
model = CNN().to(device)

In [48]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [50]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape

        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225620762
[Epoch:    2] cost = 0.0630199909
[Epoch:    3] cost = 0.046338141
[Epoch:    4] cost = 0.0374934077
[Epoch:    5] cost = 0.0314498097
[Epoch:    6] cost = 0.0261448771
[Epoch:    7] cost = 0.0217737667
[Epoch:    8] cost = 0.0182365105
[Epoch:    9] cost = 0.016359888
[Epoch:   10] cost = 0.0133685488
[Epoch:   11] cost = 0.0101897297
[Epoch:   12] cost = 0.0101049282
[Epoch:   13] cost = 0.00833414495
[Epoch:   14] cost = 0.00644074427
[Epoch:   15] cost = 0.00643952331


In [52]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
                        # 이를 axis 1방향으로 max 값을 찾는다는것
    correct_prediction = torch.argmax(prediction, 1) == Y_test # test는 숫자 정답값
                        #torch.argmax(prediction, 1) 는 피쳐값을 max인 인덱스(index1)을 뽑는것
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9878000020980835


In [53]:
# 모델 저장 
torch.save(model.state_dict(),"cnn_model.pt")

# **3-1 다시불러와서 추론 해보기기**

In [54]:
# 다시불러와서 추론 해보기
model = CNN().to(device)
model.load_state_dict(torch.load("cnn_model.pt")) 
model.eval() #평가 모드로 설정하여야 합니다. 이 과정을 거치지 않으면 일관성 없는 추론 결과가 출력

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [55]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
  
    prediction = model(X_test)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9878000020980835


In [56]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

with torch.no_grad():

    # 이미지 파일 경로 설정
    img = Image.open("./8.png")
    transform = transforms.Compose([
          transforms.Grayscale(num_output_channels=1), # RGB(3D) -> Gray(2D)
          transforms.Resize((28, 28)), # 모델 인풋에 맞게
          transforms.ToTensor(), # 토치 텐서 타입으로 맞춰줘야함
      ])
    
    img_tensor = transform(img).to(device) # [1, 28, 28]
    img_tensor = img_tensor.unsqueeze(0) # [1, 1, 28, 28]

    print(img_tensor.shape)

    prediction = model(img_tensor)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것 
    print('result:', torch.argmax(prediction, 1))

torch.Size([1, 1, 28, 28])
result: tensor([8], device='cuda:0')
